<a href="https://colab.research.google.com/github/ynoval/pdf_spell_checker/blob/main/pdf_writter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pymupdf PyPDF2 reportlab


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 44.1 MB/s eta 0:00:00


In [3]:
import fitz  # PyMuPDF
import io
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from PyPDF2 import PdfReader, PdfWriter

def extraer_texto_y_metadatos(pdf_path):
    doc = fitz.open(pdf_path)
    datos_extraidos = []

    for num_page, page in enumerate(doc):
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:
            for l in b["lines"]:
                for s in l["spans"]:
                    datos = {
                        'texto': s['text'],
                        'x': s['bbox'][0],
                        'y': letter[1] - s['bbox'][3],  # Ajuste para ReportLab
                        'tipo_de_letra': s['font'],
                        'tamano_de_letra': s['size'],
                        'pagina': num_page,
                    }
                    datos_extraidos.append(datos)
    return datos_extraidos

def crear_pdf_con_correcciones(datos_extraidos, nombre_archivo_salida):
    packet = io.BytesIO()
    c = canvas.Canvas(packet, pagesize=letter)

    pagina_actual = -1  # Inicializar con un número de página que no existe

    for item in datos_extraidos:
        if item['pagina'] > pagina_actual:
            if pagina_actual != -1:  # Evitar añadir una nueva página en la primera iteración
                c.showPage()
            pagina_actual = item['pagina']

        c.setFont("Helvetica", item['tamano_de_letra'])  # Usar Helvetica como ejemplo
        c.drawString(item['x'], item['y'], item['texto'])

    c.save()
    packet.seek(0)

    # Crear un nuevo PDF con PyPDF2
    new_pdf = PdfReader(packet)
    output = PdfWriter()

    for page in new_pdf.pages:
        output.add_page(page)

    with open(nombre_archivo_salida, "wb") as f:
        output.write(f)

# Ruta al archivo PDF original
archivo_original = "sample_data/example.pdf"

# Extraer texto y metadatos
datos_extraidos = extraer_texto_y_metadatos(archivo_original)

# Crear nuevo PDF con correcciones
nombre_archivo_salida = archivo_original.replace(".pdf", "_v2.pdf")
crear_pdf_con_correcciones(datos_extraidos, nombre_archivo_salida)




FileNotFoundError: no such file: 'sample_data/example.pdf'